In [1]:
import os

In [2]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [3]:
os.chdir("../")

In [25]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [26]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    categorical_algorithm: str
    l1_regularization: float
    l2_categorical_regularization: float
    l2_regularization: float
    max_depth: int
    num_trees: int
    target_column: str

In [27]:

from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml, create_directories

In [28]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        This function returns the ModelTrainerConfig object that contains the configuration for the model training process.

        Args:
            None

        Returns:
            ModelTrainerConfig: The ModelTrainerConfig object containing the configuration for the model training process

        """
        config = self.config.model_trainer
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            categorical_algorithm=params.categorical_algorithm,
            l1_regularization=params.l1_regularization,
            l2_categorical_regularization=params.l2_categorical_regularization,
            l2_regularization=params.l2_regularization,
            max_depth=params.max_depth,
            num_trees=params.num_trees,
            target_column=schema.name,
        )
        return model_trainer_config

In [29]:
import pandas as pd
import os
from ThyroidProject import logger
import tensorflow_decision_forests as tfdf

In [52]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        """
        trains the model using the training data and saves it to the artifacts directory
        """
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        tf_dataset = tfdf.keras.pd_dataframe_to_tf_dataset(
            train_data,label=self.config.target_column , max_num_classes=3)
        print(tf_dataset)

        model = tfdf.keras.GradientBoostedTreesModel(
            categorical_algorithm=self.config.categorical_algorithm,
            l1_regularization=self.config.l1_regularization,
            l2_categorical_regularization=self.config.l2_categorical_regularization,
            l2_regularization=self.config.l2_regularization,
            max_depth=self.config.max_depth,
            num_trees=self.config.num_trees
        )
        model.fit(tf_dataset)
        inspector = model.make_inspector()
        logger.info(
            f"The results of the model building are: {inspector.training_logs()}")
        inspector.export_to_tensorboard(os.path.join(self.config.root_dir,"tensorboard_logs"))
        model.save(os.path.join(self.config.root_dir, self.config.model_name))

In [53]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-27 18:11:49,457:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-01-27 18:11:49,459:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-27 18:11:49,461:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-01-27 18:11:49,462:INFO:common:created directory at :artifacts]
[2024-01-27 18:11:49,463:INFO:common:created directory at :artifacts/model_trainer]
[2024-01-27 18:11:49,477:WARNING:tf_logging:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.]
<_PrefetchDataset element_spec=({'age': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'sex': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'on_thyroxine': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'query_on_thyroxine': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'on_antihyroid_meds': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'sick': TensorSpec(shape=(None,), dtype=tf.str

[WARNING 24-01-27 18:11:49.5054 IST gradient_boosted_trees.cc:1886] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-27 18:11:49.5054 IST gradient_boosted_trees.cc:1897] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-01-27 18:11:49.5054 IST gradient_boosted_trees.cc:1911] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


Model trained in 0:00:00.579805
Compiling model...
Model compiled.
[2024-01-27 18:11:50,384:INFO:423033297:The results of the model building are: [TrainLog(num_trees=1, evaluation=Evaluation(num_examples=None, accuracy=0.9787610769271851, loss=0.8328334093093872, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)), TrainLog(num_trees=2, evaluation=Evaluation(num_examples=None, accuracy=0.9787610769271851, loss=0.6510120034217834, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)), TrainLog(num_trees=3, evaluation=Evaluation(num_examples=None, accuracy=0.9787610769271851, loss=0.5196457505226135, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)), TrainLog(num_trees=4, evaluation=Evaluation(num_examples=None, accuracy=0.9805309772491455, loss=0.4200752377510071, rmse=None, ndcg=None, aucs=None, auuc=None, qini=None)), TrainLog(num_trees=5, evaluation=Evaluation(num_examples=None, accuracy=0.9805309772491455, loss=0.3441726565361023, rmse=None, ndcg=None, aucs=None, auuc=No

[INFO 24-01-27 18:11:50.2621 IST kernel.cc:1233] Loading model from path /tmp/tmpirqqe2zc/model/ with prefix 56a81e0e45754deb
[INFO 24-01-27 18:11:50.2664 IST decision_forest.cc:660] Model loaded with 111 root(s), 4117 node(s), and 14 input feature(s).
[INFO 24-01-27 18:11:50.2664 IST kernel.cc:1061] Use fast generic engine


[2024-01-27 18:11:51,090:INFO:builder_impl:Assets written to: artifacts/model_trainer/model/assets]
